<a href="https://colab.research.google.com/github/ChihabEddine98/AmazonAnalysis/blob/main/AmazonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO LIST
--------


- [x] Genral collect for dataset
- [ ] Complete dataset
- [ ] Make settings in an external file
- [ ] Make settings in an external file
- [ ] Make requirement installations in another file
- [ ] Clean Data & prepocessing


# Requirements Installations

In [1]:
# Run just once 
!pip install webdriver-manager
!pip install selenium
!pip install requests
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.5MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/d

In [2]:
# imports
import os
import sys
import requests as rq
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
import unicodedata
import pickle as pk
# end imports

# Part 01  : Data Gathering

In [13]:
# Settings 
reviews_div_cls = 'review'


prod_urls = {'informatique':
              [
                {
                    'prod_id': 1,
                    'prod_url':
                    'https://www.amazon.fr/Exclusif-Microsoft-Surface-Windows-tactile/dp/B08CNLLM57/ref=sr_1_11?__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=Microsoft+Surface&qid=1618066809&s=computers&sr=1-11',
                    'rev_url':
                    'https://www.amazon.fr/Exclusif-Microsoft-Surface-Windows-tactile/product-reviews/B08CNLLM57/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                    'rev_pages': 5 ,
                    'rev_count' : 50
                
                },
                {
                    'prod_id': 2,
                    'prod_url':
                    'https://www.amazon.fr/HP-302-Pack-cartouches-Authentiques/dp/B01LXLFF6H?ref_=Oct_s9_apbd_omwf_hd_bw_bN4CkB&pf_rd_r=V2TTM35Y3ZRBSP9XFVDS&pf_rd_p=3d37f0c5-041f-5707-bec0-94d62f6af77d&pf_rd_s=merchandised-search-10&pf_rd_t=BROWSE&pf_rd_i=340858031&th=1',
                    'rev_url':
                    'https://www.amazon.fr/HP-302-Pack-cartouches-Authentiques/product-reviews/B01LXLFF6H/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                    'rev_pages': 10 ,
                    'rev_count' : 100
                
                },
               {
                   'prod_id':3,
                   'prod_url':
                   'https://www.amazon.fr/Logitech-capteur-programmables-ordinateur-portable/dp/B07GS6ZB7T/ref=sr_1_6?brr=1&dchild=1&qid=1618071105&rd=1&s=computers&sr=1-6&th=1',
                   'rev_url':
                   'https://www.amazon.fr/Logitech-capteur-programmables-ordinateur-portable/product-reviews/B07GS6ZB7T/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                   'rev_pages': 10 ,
                   'rev_count' : 100
                   
               },
               {
                   'prod_id':4,
                   'prod_url':
                   'https://www.amazon.fr/microSDXC-SanDisk-Adaptateur-Performances-Applicatives/dp/B07FCMKK5X/ref=sr_1_4?brr=1&dchild=1&qid=1618071452&rd=1&s=computers&sr=1-4&th=1',
                   'rev_url':
                   'https://www.amazon.fr/microSDXC-SanDisk-Adaptateur-Performances-Applicatives/product-reviews/B07FCMKK5X/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                   'rev_pages': 10 ,
                   'rev_count' : 100
                
                   
               }
            ],
            'cuisine':
            [
              {
                  'prod_id': 5,
                  'prod_url':
                  'https://www.amazon.fr/AmazonBasics-Batterie-Cuisine-Anti-adh%C3%A9sive-pi%C3%A8ces/dp/B07481LPMF/ref=sr_1_5?_encoding=UTF8&c=ts&dchild=1&keywords=Casseroles%2C+po%C3%AAles+et+faitouts&qid=1618067686&s=kitchen&sr=1-5&ts_id=2969505031',
                  'rev_url':
                  'https://www.amazon.fr/AmazonBasics-Batterie-Cuisine-Anti-adh%C3%A9sive-pi%C3%A8ces/product-reviews/B07481LPMF/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews',
                  'rev_pages': 5 ,
                  'rev_count' : 50
              
              }
            ]
             
}

prod_cols = ['Prod_Title','Prod_Brnd','Prod_Rate','Prod_Eval','Prod_Price','Category','Nb_Revs','Prod_ID']
rev_cols = ['Rev_Title','Rev_Rate','Rev_Bdy','Rev_Hlp','Prod_ID']


# To split settings into another file
settings_file = open("settings.pkl", "wb")
pk.dump(prod_urls, settings_file)
settings_file.close()

settings_file = open("settings.pkl", "rb")
output = pk.load(settings_file)



# End Settings

# Scrapping Product Information

In [14]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text

    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,prod_rate,prod_eval,prod_price,prod_category]))
    return result



# Scrapping Reviews Data

In [15]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        try :
          review_title = review.find('a',{'data-hook' : 'review-title'}).text
        except AttributeError:
          review_title = 'Null'
        
        try :
          review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        except AttributeError:
          review_rate = '0'
        
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        
        try :
          review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        except AttributeError:
          review_help = '0'
        
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+(prod,))

    return ds

# Testing data collect

## Setup the web-driver 

In [16]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options = options)

## Scrapping Products

In [17]:
lst_prod = [] 
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])+(prod['rev_count'],prod['prod_id'])
        lst_prod.append(product)




df_prod = pd.DataFrame(lst_prod)
df_prod.columns = prod_cols
df_prod

,Prod_Title,Prod_Brnd,Prod_Rate,Prod_Eval,Prod_Price,Category,Nb_Revs,Prod_ID
0,Pack Exclusif Microsoft Surface Pro 7 Ordinate...,Marque : Microsoft Surface,"4,7 sur 5 étoiles",3 274 évaluations,"1 195,99 €",informatique,50,1
1,HP 302 Pack de 2 Cartouches d'Encre Noire et T...,Visiter la boutique HP,"4,6 sur 5 étoiles",99 280 évaluations,"31,57 €",informatique,100,2
2,Logitech G502 HERO Souris Gamer Filaire Haute ...,Visiter la boutique Logitech G,"4,7 sur 5 étoiles",37 012 évaluations,"58,46 €",informatique,100,3
3,SanDisk Extreme Carte Mémoire MicroSDXC 128 G...,Visiter la boutique SanDisk,"4,7 sur 5 étoiles",197 001 évaluations,"18,54 €",informatique,100,4
4,Amazon Basics LFFP16027 Batterie de Cuisine An...,Visiter la boutique Amazon Basics,"4,4 sur 5 étoiles",25 856 évaluations,"60,29 €",cuisine,50,5


## Scrapping Reviews Data




In [18]:
# This function help us to generate the dataset of reviews
# and also the dataframe of several products

lst_rev = []
for category,products in prod_urls.items():
    for prod in products:
        for i in range(1,prod['rev_pages']+1):
            lst_rev = reviews_scrapper(driver,prod['prod_id'],f'{prod["rev_url"]}&pageNumber={i}',lst_rev) 

df_rev = pd.DataFrame(lst_rev)
df_rev.columns = rev_cols
df_rev.info()
 
df_rev.to_csv("Review_data.csv")
df_rev = pd.read_csv("Review_data.csv")
df_rev




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Rev_Title  400 non-null    object
 1   Rev_Rate   400 non-null    object
 2   Rev_Bdy    400 non-null    object
 3   Rev_Hlp    400 non-null    object
 4   Prod_ID    400 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 15.8+ KB


,Unnamed: 0,Rev_Title,Rev_Rate,Rev_Bdy,Rev_Hlp,Prod_ID
0,0,Surface Pro 7ème génération,"4,0 sur 5 étoiles",Je possède la Surface Pro 6 (soit la généra...,155 personnes ont trouvé cela utile,1
1,1,Du reconditionné au prix du neuf!,"1,0 sur 5 étoiles",Produit bien reçu.Par contre on achète au pr...,110 personnes ont trouvé cela utile,1
2,2,THROTTING,"2,0 sur 5 étoiles",POUR LA VERSION I5 SEULEMENT : PHENOMENE DE TH...,72 personnes ont trouvé cela utile,1
3,3,ATTENTION AU TITRE TROMPEUR !,"2,0 sur 5 étoiles",Le titre fait référence à un Surface Pro 7 ...,98 personnes ont trouvé cela utile,1
4,4,C'est bien une version i7,"5,0 sur 5 étoiles",Contrairement au commentaire disant que c'est ...,44 personnes ont trouvé cela utile,1
...,...,...,...,...,...,...
395,395,Que ça va bien sûr les plaques à induction,"5,0 sur 5 étoiles",Super au delà de nos espérances belle batter...,0,5
396,396,Bon rapport qualité prix,"5,0 sur 5 étoiles",Pour le prix c'est nickelA voir sur la durée ...,0,5
397,397,muy bien!,"5,0 sur 5 étoiles",muy bien!,0,5
398,398,La solidarité et se lave facilement n accroch...,"3,0 sur 5 étoiles",Bon produit solide mais il manque 3 ustensiles...,0,5
